<a href="https://colab.research.google.com/github/rajeshmore1/Tasks---Python/blob/main/Assignment_Solution_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <u> Use only those concepts which have been taught till now </u>

# <b><u> Problem 1: </u> Which actor has acted in the most number of movies? Deep dive into the movies, genres and profits corresponding to this actor. </b>

In [ ]:
# Your code here
import pandas as pd
import numpy as np
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Data Wrangling/imdb_data.csv')
df.head(2)

In [ ]:
df['profit'] = df.revenue - df.budget
df['profit'].replace([np.inf, -np.inf], 0, inplace=True) 
pd.options.display.float_format = '{:.4f}'.format
df.profit

In [ ]:
df['new_cast'] = df.cast.fillna('[]').apply(ast.literal_eval)
df['new_genre'] = df.genres.fillna('[]').apply(ast.literal_eval)

df['new_cast'] = df.new_cast.apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
df['new_genre'] = df.new_genre.apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])
df.head(2)

In [ ]:
new_df = df.explode('new_cast')
top_actors = pd.DataFrame(new_df.new_cast.value_counts()).reset_index()
top_actors.columns = ['Actors','Number Of Movies']
top_actors.head(3)

In [ ]:
#printing genre and mean
grouped_actors = new_df.groupby('new_cast').agg({'new_genre':'sum','profit':'mean'}).reset_index()
grouped_actors.loc[(grouped_actors.new_cast=='Robert De Niro') | (grouped_actors.new_cast=='Samuel L. Jackson') | (grouped_actors.new_cast=='Morgan Freeman')].rename({'profit':'mean_profit'},axis=1)

# <b><u> Problem 2 :</u> Top 3 directors prefer which actors the most?

In [ ]:
# Your code here 
def find_directors(li):
  #print(li)
  job_iterator = 0

  if type(li)!= float:
    li = ast.literal_eval(li)
    while True:
      if li[job_iterator]['job']!='Director':
        job_iterator+=1
      else:
        return li[job_iterator]['name']

 
 
df['director_name'] = df.apply(lambda x: find_directors(x['crew']),axis = 1)
grouped_pop = df.groupby('director_name')['popularity'].mean().sort_values(ascending = False).reset_index()
grouped_pop.head(3)



In [ ]:
pop_dir_actors_1 = list(df[df['director_name'] == grouped_pop['director_name'][0]]['new_cast'])[0]
pop_dir_actors_2 = list(df[df['director_name'] == grouped_pop['director_name'][1]]['new_cast'])[0]
pop_dir_actors_3 = list(df[df['director_name'] == grouped_pop['director_name'][2]]['new_cast'])[0]

In [ ]:
#actors prefered by tim miller
pop_dir_actors_1

In [ ]:
#actors prefered by James Gunn
pop_dir_actors_2 

In [ ]:
#prefered by Edgar Wright	
pop_dir_actors_3

# <b><u>Problem 3 :</u> Make a table of all the producers and directors of each movie. Find the top 3 producers who have produced movies with the highest average RoI.

In [ ]:
movie producers directors

movie1 [p1, p2] [d1, d2]
movie2 [p3, p4] [d3, d4]

movie producers roi

movie1 p1 12
movie1 p2 13
movie2 p3 14
movie2 p4 15

In [ ]:
producer   movies            average_roi
p1.    [m1, m2, m3]    avg_roi(m1, m2, m3)
p2     [m2, m4, m5]    avg_roi(m2, m4, m5)

In [ ]:
imdb_df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,profit,roi
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,-1685349,-12.038207
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,55149435,137.873588
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,9792000,296.727273
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,14800000,1233.333333
4,5,NaN,19500000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970,-15576030,-79.877077


In [ ]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [ ]:
imdb_df_no_na_crew = imdb_df[~imdb_df['crew'].isna()]

In [ ]:
len(imdb_df_no_na_crew)

2984

In [ ]:
def get_producers_and_directors(string_rep_of_crew_list):
  try:
    crew_list = literal_eval(string_rep_of_crew_list)
    producer_list = []
    director_list = []
    for elem in crew_list:
      if elem['job']=='Producer':
        producer_list.append(elem['name'])
      if elem['job']=='Director':
        director_list.append(elem['name'])
    return producer_list, director_list
  except:
    return [], []

In [ ]:
get_producers_and_directors(imdb_df_no_na_crew.loc[1, 'crew'])

(['Whitney Houston', 'Mario Iscovich', 'Debra Martin Chase'],
 ['Garry Marshall'])

In [ ]:
imdb_df[['producers', 'directors']] = imdb_df.apply(lambda s: get_producers_and_directors(s['crew']), axis=1, result_type="expand")
#imdb_df[['pnd']] = imdb_df.apply(lambda s: get_producers_and_directors(s['crew']), axis=1)




In [ ]:
imdb_df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,profit,roi,producers,directors,pnd
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,-1685349,-12.038207,[Andrew Panay],[Steve Pink],"([Andrew Panay], [Steve Pink])"
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,55149435,137.873588,"[Whitney Houston, Mario Iscovich, Debra Martin...",[Garry Marshall],"([Whitney Houston, Mario Iscovich, Debra Marti..."
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,9792000,296.727273,"[David Lancaster, Michel Litvak, Jason Blum, H...",[Damien Chazelle],"([David Lancaster, Michel Litvak, Jason Blum, ..."
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,14800000,1233.333333,[Sujoy Ghosh],[Sujoy Ghosh],"([Sujoy Ghosh], [Sujoy Ghosh])"
4,5,NaN,19500000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970,-15576030,-79.877077,[],[Jong-seok Yoon],"([], [Jong-seok Yoon])"


In [ ]:
{
    'p1':{'movies':['m1, m2, m3'], 'avg_roi':23},
 'p2':{}

}

In [ ]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [ ]:
producer_dict = {}
for index, row in imdb_df.iterrows():
  for producer in row['producers']:
    if producer not in producer_dict.keys():
      # add producer in dict
      producer_dict[producer] = [row['roi']]
    else:
      # update the movie for this producer
      value = producer_dict[producer]
      value.append(row['roi'])
      producer_dict[producer] = value

In [ ]:
#data = {'row_1': [3, 2, 1, 0], 'row_2': ['a', 'b', 'c', 'd']}
pd.DataFrame.from_dict(producer_dict, orient='index').mean(axis=1).reset_index().rename(columns={'index':'producer', 0:'average_roi'}).sort_values('average_roi', ascending=False)

,producer,average_roi
2635,Michael White,9307.693250
1526,Gregg Hoffman,8559.305750
3205,Nelson Greaves,6188.209000
2842,Howard G. Minsky,6100.000000
1264,Norman Spencer,5458.946615
...,...,...
1804,Arthur H. Nadel,-99.999964
1805,Lou Scheimer,-99.999964
1970,Elia Kazan,-99.999974
601,Pushpa Kandaswamy,-99.999985
